<a href="https://colab.research.google.com/github/dorian-goueytes/L2-P-M2_motor_variability/blob/main/TD_motor_variability_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ast

In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
mouse_x_error, mouse_y_error, abs_click_pos_error, rel_click_pos_error, RT_error, acc_error =  [], [], [],[],[],[]
mouse_x_reward, mouse_y_reward, abs_click_pos_reward,rel_click_pos_reward, RT_reward, acc_reward =  [], [], [],[],[],[]
all_correct, all_RT = [],[]
target_pos = [[-0.8,-0.8/(1920/1080)],[-0.8,0.8/(1920/1080)],[0.8,0.8/(1920/1080)],[0.8,-0.8/(1920/1080)]]

for trial in range(0, len(data['trial_type'])):
  if trial >=20:
    all_correct.append(data["correct"][trial])
    all_RT.append(data["RT"][trial])
    if data['trial_type'][trial] == "reward_feedback":
      x = ast.literal_eval(data['mouse_3.x'][trial])
      y = ast.literal_eval(data['mouse_3.y'][trial])
      click = ast.literal_eval(data["click_pos"][trial])
      target = int(data['Target'][trial])
      mouse_x_reward.append(np.array(x))
      mouse_y_reward.append(np.array(y)*(1920/1080))
      abs_click_pos_reward.append(click)
      rel_click_pos_reward.append(np.array(click)-np.array(target_pos[target]))
      acc_reward.append(data["correct"][trial])
      RT_reward.append(data["RT"][trial])

    if data['trial_type'][trial] == "error_feedback":
      x = ast.literal_eval(data['mouse_3.x'][trial])
      y = ast.literal_eval(data['mouse_3.y'][trial])
      click = ast.literal_eval(data["click_pos"][trial])
      target = int(data['Target'][trial])
      mouse_x_error.append(np.array(x))
      mouse_y_error.append(np.array(y)*(1920/1080))
      abs_click_pos_error.append(click)
      rel_click_pos_error.append(np.array(click)-np.array(target_pos[target]))
      acc_error.append(data["correct"][trial])
      RT_error.append(data["RT"][trial])

# Analyse des performances

## Analyses des performances en fonction du types d'essai

In [ ]:
perf_globale = len(np.nonzero(all_correct)[0])/len(all_correct)*100
perf_reward = len(np.nonzero(acc_reward)[0])/len(acc_reward)*100
perf_error = len(np.nonzero(acc_error)[0])/len(acc_error)*100
print('Performance globale : ', perf_globale, "% correct")
print('Performance essais reward : ', perf_reward, "% correct")
print('Performance essais erreur : ', perf_error, "% correct")
print()

plt.figure()
plt.plot([1, 2], [perf_reward, perf_error], color = 'b')#, marker='o', linestyle='-')
plt.scatter(1, perf_reward, label="Reward", color='b')
plt.scatter(2, perf_error, label="Error", color='b')

plt.ylim(0, 100)
plt.xticks([1, 2], ["Reward", "Error"])  # Only two x-ticks
plt.xlabel('Condition')
plt.ylabel("Pourcentage de click dans la cible")
plt.title("Evolution des performances en fonction des essais")
plt.tight_layout()
plt.show()

## Analyse des performances au cours du temps

In [ ]:
def split_into_chunks(lst, chunk_size):
    """Splits a list into chunks of given size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

fig, ax = plt.subplots(1,1)
time_reward = split_into_chunks(acc_reward, 10)
time_error = split_into_chunks(acc_error, 10)

temp_reward = []

for i in time_reward:
  if len(i) >=5:
    temp_reward.append(len(np.nonzero(i)[0])/len(i)*100)
ax.plot(np.linspace(0, 40,4),temp_reward, color = 'g', marker = '^', label = 'Reward trials')

temp_error = []
for i in time_error:
  if len(i) >=5:
    temp_error.append(len(np.nonzero(i)[0])/len(i)*100)
ax.plot(np.linspace(0, 40,4), temp_error, color = 'b', marker = 's', label = 'Error trials')
ax.legend()
ax.set_xlabel('Essais effectués')
ax.set_ylabel('"Pourcentage de click dans la cible"')
plt.title("Evolution des performances en fonction du nombre d'essais")
plt.tight_layout()
plt.show()

# Exploration des temps de réactions

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2)
ax1.hist(all_RT, bins = 8, label = 'All trials')
ax1.set_ylabel("Temps de réaction")
ax1.set_xlabel("Proportion d'essais")
ax1.set_title("Tous les essais")
ax1.legend()
ax2.hist(RT_reward, bins = 8, histtype = 'step',color = 'g', label = 'Reward trials')
ax2.axvline(np.mean(RT_reward), color = 'g', linestyle = '--')
ax2.hist(RT_error, bins = 8, histtype = 'step', label = 'Error trials')
ax2.axvline(np.mean(RT_error), linestyle = '--')
ax2.legend()
ax2.set_title("Séparé selon les conditions")
ax2.set_ylabel("Temps de réaction")
ax2.set_xlabel("Proportion d'essais")
plt.suptitle("Distribution des temps de réactions")

plt.tight_layout()
plt.show()

# Comparaison des trajectoires

In [ ]:
fig,((ax1,ax2), (ax3,ax4)) = plt.subplots(2,2,figsize = (8,8))
circle1 = plt.Circle((-0.8, -0.8), 0.15, edgecolor='r', facecolor = "None")
circle2 = plt.Circle((-0.8, 0.8),0.15, edgecolor='r', facecolor = "None")
circle3 = plt.Circle((0.8, 0.8),0.15, edgecolor='r', facecolor = "None")
circle4 = plt.Circle((0.8, -0.8),0.15, edgecolor='r', facecolor = "None")

for i in range(0, len(mouse_x_reward)):
  ax1.plot(mouse_x_reward[i], mouse_y_reward[i], color = 'g')
ax1.add_patch(circle1)
ax1.add_patch(circle2)
ax1.add_patch(circle3)
ax1.add_patch(circle4)
ax1.set_xlabel("Position de la souris (Horiziontale)")
ax1.set_ylabel("Position de la souris (Verticale)")
ax1.set_title('Trajectoires pour les essais reward')

for i in range(0, len(mouse_x_error)):
  ax2.plot(mouse_x_error[i], mouse_y_error[i], color = 'b')
circle1 = plt.Circle((-0.8, -0.8), 0.15, edgecolor='r', facecolor = "None")
circle2 = plt.Circle((-0.8, 0.8),0.15, edgecolor='r', facecolor = "None")
circle3 = plt.Circle((0.8, 0.8),0.15, edgecolor='r', facecolor = "None")
circle4 = plt.Circle((0.8, -0.8),0.15, edgecolor='r', facecolor = "None")
ax2.add_patch(circle1)
ax2.add_patch(circle2)
ax2.add_patch(circle3)
ax2.add_patch(circle4)
ax2.set_xlabel("Position de la souris (Horiziontale)")
ax2.set_ylabel("Position de la souris (Verticale)")
ax2.set_title('Trajectoires pour les essais error')


circle1 = plt.Circle((-0.8, -0.8), 0.15, edgecolor='r', facecolor = "None")
circle2 = plt.Circle((-0.8, 0.8),0.15, edgecolor='r', facecolor = "None")
circle3 = plt.Circle((0.8, 0.8),0.15, edgecolor='r', facecolor = "None")
circle4 = plt.Circle((0.8, -0.8),0.15, edgecolor='r', facecolor = "None")

ax3.scatter(abs_click_pos_reward[0][0], abs_click_pos_reward[0][1]*(1920/1080), color = 'g', alpha =1, label = 'Reward Trials')
ax4.scatter(abs_click_pos_reward[0][0], abs_click_pos_reward[0][1]*(1920/1080), color = 'g', alpha = 0.2, label = 'Reward Trials')

for i in range(0, len(abs_click_pos_reward)):
  ax3.scatter(abs_click_pos_reward[i][0], abs_click_pos_reward[i][1]*(1920/1080), color = 'g', alpha = 1)
  ax4.scatter(abs_click_pos_reward[i][0], abs_click_pos_reward[i][1]*(1920/1080), color = 'g', alpha = 0.2)


ax3.add_patch(circle1)
ax3.add_patch(circle2)
ax3.add_patch(circle3)
ax3.add_patch(circle4)

circle1 = plt.Circle((-0.8, -0.8), 0.15, edgecolor='r', facecolor = "None")
circle2 = plt.Circle((-0.8, 0.8),0.15, edgecolor='r', facecolor = "None")
circle3 = plt.Circle((0.8, 0.8),0.15, edgecolor='r', facecolor = "None")
circle4 = plt.Circle((0.8, -0.8),0.15, edgecolor='r', facecolor = "None")
ax4.scatter(abs_click_pos_error[0][0], abs_click_pos_error[0][1]*(1920/1080), color = 'b', alpha = 1, label = 'Error trials')
ax3.scatter(abs_click_pos_error[0][0], abs_click_pos_error[0][1]*(1920/1080), color = 'b', alpha = 0.2, label = 'Error trials')
for i in range(0, len(abs_click_pos_error)):
  ax4.scatter(abs_click_pos_error[i][0], abs_click_pos_error[i][1]*(1920/1080), color = 'b', alpha = 1)
  ax3.scatter(abs_click_pos_error[i][0], abs_click_pos_error[i][1]*(1920/1080), color = 'b', alpha = 0.2)
ax4.add_patch(circle1)
ax4.add_patch(circle2)
ax4.add_patch(circle3)
ax4.add_patch(circle4)
ax3.legend()
ax4.legend()
plt.suptitle("Trajectoires et positions finales en fonction des conditions")

plt.tight_layout()
plt.show()

# Distribution des positions finales

In [ ]:
from matplotlib.patches import Ellipse
fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize = (18,6))
circle1 = plt.Circle((0, 0), 0.15, edgecolor='r', facecolor = "None")
circle2 = plt.Circle((0, 0), 0.15, edgecolor='r', facecolor = "None")
circle3 = plt.Circle((0, 0), 0.15, edgecolor='r', facecolor = "None")

for i in range(0, len(rel_click_pos_reward)):
  ax1.scatter(rel_click_pos_reward[i][0], rel_click_pos_reward[i][1], color = 'g')
  ax3.scatter(rel_click_pos_reward[i][0], rel_click_pos_reward[i][1], color = 'g')
ax1.add_patch(circle1)
ax2.add_patch(circle2)
ax3.add_patch(circle3)
ax1.axvline(0, color = 'k', linestyle = '--')
ax1.axhline(0, color = 'k', linestyle = '--')
x_coord = []
y_coord = []
for i in rel_click_pos_reward:
  x_coord.append(i[0])
  y_coord.append(i[1])
mean_x = np.mean(x_coord)
mean_y = np.mean(y_coord)
std_x = np.std(x_coord)
std_y = np.std(y_coord)
ax1.scatter(mean_x, mean_y, marker = '^', color = 'g', s = 200, label = 'Moyenne Essais Reward')
ax3.scatter(mean_x, mean_y, marker = '^', color = 'g', s = 200, label = 'Moyenne Essais Reward')

ellipse = Ellipse(xy=(mean_x, mean_y),width=std_x*2,height=std_y*2,angle=0,facecolor="none",edgecolor="g", linestyle = '--')
ax1.add_patch(ellipse)
ellipse = Ellipse(xy=(mean_x, mean_y),width=std_x*2,height=std_y*2,angle=0,facecolor="none",edgecolor="g", linestyle = '--')
ax3.add_patch(ellipse)


for i in range(0, len(rel_click_pos_error)):
  ax2.scatter(rel_click_pos_error[i][0], rel_click_pos_error[i][1], color = 'b')
  ax3.scatter(rel_click_pos_error[i][0], rel_click_pos_error[i][1], color = 'b')
  #ax.scatter(mouse_x_error[i][-1], mouse_y_error[i][-1], color = 'b', marker = '^')
ax2.axvline(0, color = 'k', linestyle = '--')
ax2.axhline(0, color = 'k', linestyle = '--')

x_coord = []
y_coord = []
for i in rel_click_pos_error:
  x_coord.append(i[0])
  y_coord.append(i[1])
mean_x = np.mean(x_coord)
mean_y = np.mean(y_coord)
std_x = np.std(x_coord)
std_y = np.std(y_coord)
ax2.scatter(mean_x, mean_y, marker = 's', color = 'b', s = 150, label = 'Moyenne Essais Erreur')
ax3.scatter(mean_x, mean_y, marker = 's', color = 'b', s = 150, label = 'Moyenne Essais Erreur')

ellipse = Ellipse(xy=(mean_x, mean_y),width=std_x*2,height=std_y*2,angle=0,facecolor="none",edgecolor="b", linestyle = '--')
ax2.add_patch(ellipse)
ellipse = Ellipse(xy=(mean_x, mean_y),width=std_x*2,height=std_y*2,angle=0,facecolor="none",edgecolor="b", linestyle = '--')
ax3.add_patch(ellipse)
ax3.axvline(0, color = 'k', linestyle = '--')
ax3.axhline(0, color = 'k', linestyle = '--')

ax1.legend()
ax1.set_xlabel("Position du click par rapport aux cibles (Hor.)")
ax1.set_ylabel("Position du click par rapport aux cibles (Hor.)")
ax2.legend()
ax2.set_xlabel("Position du click par rapport aux cibles (Hor.)")
ax2.set_ylabel("Position du click par rapport aux cibles (Hor.)")
ax3.legend()
ax3.set_xlabel("Position du click par rapport aux cibles (Hor.)")
ax3.set_ylabel("Position du click par rapport aux cibles (Hor.)")

plt.suptitle("Distribution des réponses autour du centre des cibles")
ax1.set_title("Distribution des essais reward")
ax2.set_title("Distribution des essais error")
ax3.set_title("Distributions superposées essais reward et error")
plt.tight_layout()
plt.show()